### Metagenomics

In [33]:
import os
import pandas as pd
import qiime2 as q2
import requests

from qiime2 import Visualization

data_dir = 'CE'
    
%matplotlib inline

In [2]:
def fetch_ipath(ids: list, img_output_path: str, verbose: bool = False):
    """Fetches a enriched pathways map from iPATH3 for given IDs."""
    url = 'https://pathways.embl.de/mapping.cgi'
    
    # remove colon from EC names
    if ':' in ids[0]:
        ids = [x.replace(':', '') for x in ids]
    
    if verbose:
        print(f'Fetching iPATH3 diagram for ids: {ids}')
    params = {
        'default_opacity': 0.6,
        'export_type': 'svg',
        'selection': '\n'.join(ids)
    }   
    response = requests.get(url=url, params=params)
    
    with open(img_output_path, 'wb') as img:
        img.write(response.content)

In [3]:
picrust_env = '/opt/conda/envs/picrust2/bin'

#### Functional inference

As this step would take very long to run, all the artifacts that would be generated in this step are ready to download in the next command (Download files).

In [26]:
#%%script env picrust_env="$picrust_env" data_dir="$data_dir" bash

#export PATH=$picrust_env:$PATH

#$picrust_env/qiime picrust2 full-pipeline \
 #   --i-seq $data_dir/dada2_rep_set_filtered.qza \
  #  --i-table $data_dir/dada2_table_align_filtered.qza \
   # --output-dir $data_dir/picrust2_results \
    #--p-placement-tool sepp \
    #--p-threads 2 \
    #--p-hsp-method pic \
    #--p-max-nsti 2 

##### Download files

In [5]:
! wget -nv -O $data_dir/picrust2_results/metagenomics.zip 'https://polybox.ethz.ch/index.php/s/9IoT5okOckQUCl5/download'

2022-11-23 10:40:45 URL:https://polybox.ethz.ch/index.php/s/9IoT5okOckQUCl5/download [47323823] -> "CE/picrust2_results/metagenomics.zip" [1]


In [6]:
! unzip -q $data_dir/picrust2_results/metagenomics.zip -d $data_dir
! rm $data_dir/picrust2_results/metagenomics.zip

##### visualize metadata

In [38]:
metadata = pd.read_csv(f'{data_dir}/food-metadata.tsv', sep='\t', header=0, index_col=0)

In [3]:
! qiime metadata tabulate \
    --m-input-file $data_dir/food-metadata.tsv \
    --o-visualization $data_dir/food-metadata.qzv

Saved Visualization to: CE/food-metadata.qzv


In [34]:
Visualization.load(f'{data_dir}/food-metadata.qzv')

<visualization: Visualization uuid: b7f93fda-aff6-4163-b538-49fb64f5b15a>

Then the unfiltered picrust2 results were loaded and viewed as dataframes. after that the generated tables are quickly checked by using the df.head(3) function.

In [32]:
ko = q2.Artifact.load(f'{data_dir}/picrust2_results/ko_metagenome.qza').view(pd.DataFrame)
ec = q2.Artifact.load(f'{data_dir}/picrust2_results/ec_metagenome.qza').view(pd.DataFrame)
pa = q2.Artifact.load(f'{data_dir}/picrust2_results/pathway_abundance.qza').view(pd.DataFrame)

In [29]:
ko.head(3)

,K00001,K00002,K00003,K00004,K00005,K00007,K00008,K00009,K00010,K00011,...,K19776,K19777,K19778,K19779,K19780,K19784,K19785,K19788,K19789,K19791
11488.CSB279,20.870000,0.000000,29.869996,19.088461,11.732050,0.000000,18.000000,18.870000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
11488.CSB280,4710.087735,0.883903,3087.591228,570.973332,598.687396,165.710000,2901.450636,1056.542353,951.836496,0.0,...,165.710000,0.0,0.0,0.0,0.0,282.312795,0.0,0.0,165.71,0.0
11488.CSB281,18839.163176,3.409240,14063.556301,2020.855062,1447.226731,112.020927,13866.909927,7162.240350,3390.350956,0.0,...,112.020927,0.0,0.0,0.0,0.0,534.467339,0.0,0.0,237.86,0.0


In [30]:
ec.head(3)

,EC:1.1.1.1,EC:1.1.1.10,EC:1.1.1.100,EC:1.1.1.101,EC:1.1.1.102,EC:1.1.1.103,EC:1.1.1.105,EC:1.1.1.107,EC:1.1.1.108,EC:1.1.1.11,...,EC:6.4.1.8,EC:6.5.1.1,EC:6.5.1.2,EC:6.5.1.3,EC:6.5.1.4,EC:6.5.1.5,EC:6.5.1.6,EC:6.5.1.7,EC:6.6.1.1,EC:6.6.1.2
11488.CSB279,65.976567,2.732050,118.609739,0.0,0.000000,8.267950,0.0,0.0,18.000000,0.000000,...,0.0,0.000000,29.870000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000
11488.CSB280,21375.766822,0.000000,25027.534329,0.0,1.233809,217.384043,0.0,2008.0,2418.591401,165.710000,...,6024.0,6645.273765,3604.834077,434.392199,1.847061e-12,0.0,2249.739494,2249.739494,4385.110009,2254.049543
11488.CSB281,87446.728701,0.017552,101400.676468,0.0,0.000000,391.650232,0.0,8139.0,10094.037269,112.020927,...,24417.0,26331.128764,14365.575900,3813.974857,4.250465e-12,0.0,8801.180360,8801.180360,17249.749391,8836.278806


In [31]:
pa.head(3)

,1CMET2-PWY,3-HYDROXYPHENYLACETATE-DEGRADATION-PWY,AEROBACTINSYN-PWY,ALL-CHORISMATE-PWY,ANAEROFRUCAT-PWY,ANAGLYCOLYSIS-PWY,ARG+POLYAMINE-SYN,ARGDEG-PWY,ARGORNPROST-PWY,ARGSYN-PWY,...,THISYN-PWY,THREOCAT-PWY,THRESYN-PWY,TRNA-CHARGING-PWY,TRPSYN-PWY,TYRFUMCAT-PWY,UBISYN-PWY,UDPNAGSYN-PWY,VALDEG-PWY,VALSYN-PWY
11488.CSB279,29.087959,0.000000,0.0,0.000000,32.959133,39.774215,0.000000,0.000000,3.113013,27.818255,...,0.000000,0.000000,30.567475,26.785641,33.139028,0.000000,13.567483,32.052976,0.0,40.751255
11488.CSB280,4897.933964,511.684352,0.0,900.991069,4305.126894,4163.846656,1156.690606,323.661369,2554.919784,3839.894544,...,1721.375952,391.171377,3825.459102,2781.586473,3451.307117,802.594678,369.631491,3593.319358,0.0,5636.811638
11488.CSB281,20378.996367,767.883225,0.0,1925.233525,18908.454250,17719.000907,2168.025574,330.649935,8489.141475,16836.100159,...,3062.271909,600.324140,16594.619391,12912.178157,15623.688919,1730.403876,740.066897,16022.014999,0.0,24485.801019


The tables now contain information about different levels of the functional profiles:

1. `ko` table: columns represent KEGG orthologs, as indicated by their names (e.g., **K**19777)
2. `ec` table: columns represent enzymes, as indicated by the Enzyme Commission numbers (e.g., **EC**:1.1.1.108)
3. `pa` table: columns represent entire pathways using the MetaCyc classification (e.g., ANAGLYCOLYSIS-PWY)

Feature table was merged with the pasteurization column from the metadata.

In [39]:
ko_meta = ko.merge(metadata[['pasteurized']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['pasteurized']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['pasteurized']], left_index=True, right_index=True)

In [40]:
ko_meta_avg = ko_meta.groupby('pasteurized').mean()
ec_meta_avg = ec_meta.groupby('pasteurized').mean()
pa_meta_avg = pa_meta.groupby('pasteurized').mean()

In [41]:
ko_meta_avg.head()

,K00001,K00002,K00003,K00004,K00005,K00007,K00008,K00009,K00010,K00011,...,K19776,K19777,K19778,K19779,K19780,K19784,K19785,K19788,K19789,K19791
pasteurized,,,,,,,,,,,,,,,,,,,,,
N,45212.003827,185.908689,37531.219524,14012.060133,5540.705731,39.233664,26725.283658,8666.802792,4928.603070,0.000004,...,206.156283,198.177837,221.580649,1.507643e-38,8.650003,3140.075768,4.876630e-125,9.019276e-55,2227.031159,0.520927
Y,48813.259252,322.071220,49677.769682,21221.676186,10779.027223,237.477202,36895.098419,16783.979802,18479.601949,0.000676,...,431.210745,204.207785,236.545723,8.246362e-38,12.900228,10312.811629,6.062880e-125,1.121323e-54,6174.098661,10.168494


In [42]:
# find top x% of the most abundant KOs, ECs and pathways in each sample type

def find_most_abundant(df: pd.DataFrame, frac):
    if 0 < frac < 1:
        frac = int(frac * len(df.columns))
    print(f'Saving {frac} most abundant features...')
    most_abundant = {
        smp: df.loc[smp, :].sort_values(ascending=False)[:frac]
        for smp in df.index
    }
    return most_abundant

ko_most_abundant = find_most_abundant(ko_meta_avg, 0.01)
ec_most_abundant = find_most_abundant(ec_meta_avg, 0.03)
pa_most_abundant = find_most_abundant(pa_meta_avg, 5)

Saving 101 most abundant features...
Saving 84 most abundant features...
Saving 5 most abundant features...


Printing the most abundant KOs:

In [15]:
print(f'10 most abundant KOs in the treatment group are: {ko_most_abundant["Y"].index[:10].tolist()}\n'
      f'10 most abundant KOs in the non-treatment group are: {ko_most_abundant["N"].index[:10].tolist()}\n')

10 most abundant KOs in the treatment group are: ['K01990', 'K02015', 'K01992', 'K00059', 'K03088', 'K02529', 'K02016', 'K02013', 'K00626', 'K07090']
10 most abundant KOs in the non-treatment group are: ['K01990', 'K01992', 'K02015', 'K03088', 'K00059', 'K02529', 'K00626', 'K02016', 'K02013', 'K00666']



In [18]:
for smp in ko_most_abundant.keys():
    fetch_ipath(ko_most_abundant[smp].index.tolist(), f'{data_dir}/kos_{smp}.svg')
    fetch_ipath(ec_most_abundant[smp].index.str.replace(':', '').tolist(), f'{data_dir}/ecs_{smp}.svg')

Nothing too interesting was seen, so let's continue with enriched pathways.

Printing the most abundant enriched pathways:

In [44]:
print(f'4 most abundant pathways in the treatment group are: {pa_most_abundant["Y"].index[:4].tolist()}\n'
      f'4 most abundant pathways in the non-treatment group are: {pa_most_abundant["N"].index[:4].tolist()}\n')

4 most abundant pathways in the treatment group are: ['PWY-7111', 'PWY-3781', 'PWY-5101', 'NONOXIPENT-PWY']
4 most abundant pathways in the non-treatment group are: ['PWY-3781', 'PWY-7111', 'PWY-5101', 'NONOXIPENT-PWY']



In [8]:
! qiime composition add-pseudocount \
    --i-table $data_dir/picrust2_results/pathway_abundance.qza \
    --o-composition-table $data_dir/picrust2_results/pathway_abundance_differences.qza

Saved FeatureTable[Composition] to: CE/picrust2_results/pathway_abundance_differences.qza


Ancom, to look at whether there are significant differences in the pathways of pasteruized verus non-pasteruized cheeses.

In [47]:
! qiime composition ancom \
    --i-table $data_dir/picrust2_results/pathway_abundance_differences.qza \
    --m-metadata-file $data_dir/food-metadata.tsv \
    --m-metadata-column pasteurized \
    --p-transform-function log \
    --o-visualization $data_dir/pa_ancom_pasteurized.qzv

Saved Visualization to: CE/pa_ancom_pasteurized.qzv


In [2]:
Visualization.load(f'{data_dir}/pa_ancom_pasteurized.qzv')

<visualization: Visualization uuid: 51d891ea-a130-4576-b4ad-9fa615e19bdb>

three significant different pathways were found for pasteurized vs non pasteurized: 
- GLUCOSE1PMETAB-PWY	with W = 465 -> more abundant in pasteurized
- PWY0-1533	with W = 420 -> more abundant in pasteurized
- PWY-6397	with W = 420 -> more abundant in non-pasteurized

Now we want to compare bloomy and washed rindtypes. As the rindtype column has more than two classes we first need to filter the third class (natural) out.

In [20]:
! qiime feature-table filter-samples \
    --i-table $data_dir/picrust2_results/pathway_abundance.qza \
    --m-metadata-file $data_dir/food-metadata.tsv \
    --p-where "[rindtype]='washed' OR [rindtype]='bloomy'" \
    --o-filtered-table $data_dir/picrust2_results/pathway_abundance_washed_bloomy.qza

Saved FeatureTable[Frequency] to: CE/picrust2_results/pathway_abundance_washed_bloomy.qza


In [21]:
pawb = q2.Artifact.load(f'{data_dir}/picrust2_results/pathway_abundance_washed_bloomy.qza').view(pd.DataFrame)

In [22]:
pawb.head()

,1CMET2-PWY,3-HYDROXYPHENYLACETATE-DEGRADATION-PWY,AEROBACTINSYN-PWY,ALL-CHORISMATE-PWY,ANAEROFRUCAT-PWY,ANAGLYCOLYSIS-PWY,ARG+POLYAMINE-SYN,ARGDEG-PWY,ARGORNPROST-PWY,ARGSYN-PWY,...,THISYN-PWY,THREOCAT-PWY,THRESYN-PWY,TRNA-CHARGING-PWY,TRPSYN-PWY,TYRFUMCAT-PWY,UBISYN-PWY,UDPNAGSYN-PWY,VALDEG-PWY,VALSYN-PWY
11488.CSB290,12637.890024,906.965037,0.000000,0.000000,11934.402822,11063.274587,663.536056,0.000000,5593.865049,10377.259212,...,1893.451621,0.00000,9614.696886,4609.063843,9756.335402,10.294863,288.190037,9883.616193,0.0,17027.740368
11488.CSB291,9812.885169,1181.754274,0.000000,0.000000,9908.779207,9114.048202,902.283025,0.000000,6031.125195,8649.332999,...,2392.746116,0.00000,7949.604075,4321.629940,8297.595173,0.000000,346.755855,8312.001306,0.0,15706.753487
11488.CSB292,10233.505770,767.084806,0.000000,0.000000,9860.887213,9155.356721,556.104221,0.000000,4472.335197,8572.653613,...,1604.466674,0.00000,7986.461627,3470.535997,8192.541574,0.000000,252.286194,8193.325410,0.0,14055.001076
11488.CSB299,925.653823,188.904398,0.297164,21.057654,1529.206959,1616.659405,154.361302,35.618171,762.807619,1304.051567,...,64.715543,0.01318,1419.361116,1169.957038,1557.423974,184.096137,117.397328,1433.639443,0.0,2084.415143
11488.CSB300,3071.639373,491.933085,0.460374,599.410248,3598.444000,3986.662395,1313.961133,665.789745,2106.527589,3760.512195,...,662.562468,0.00000,3775.423286,3137.012254,4242.688306,1952.066295,1418.647685,3790.448335,0.0,5404.696700


In [23]:
! qiime composition add-pseudocount \
    --i-table $data_dir/picrust2_results/pathway_abundance_washed_bloomy.qza \
    --o-composition-table $data_dir/picrust2_results/pathway_abundance_differences_washed_bloomy.qza

Saved FeatureTable[Composition] to: CE/picrust2_results/pathway_abundance_differences_washed_bloomy.qza


In [24]:
! qiime composition ancom \
    --i-table $data_dir/picrust2_results/pathway_abundance_differences_washed_bloomy.qza \
    --m-metadata-file $data_dir/food-metadata.tsv \
    --m-metadata-column rindtype \
    --p-transform-function log \
    --o-visualization $data_dir/pa_ancom_rindtype_bloomy_vs_washed.qzv

Saved Visualization to: CE/pa_ancom_rindtype_bloomy_vs_washed.qzv


In [46]:
Visualization.load(f'{data_dir}/pa_ancom_rindtype_bloomy_vs_washed.qzv')

<visualization: Visualization uuid: 6a23a8db-6bd8-456a-be44-c447fc517850>

For washed vs bloomy the following significantly different pathways were found:
- PWY-6397	with W = 451 -> more abundant in washed
- PWY0-1338	with W = 442 -> more abundant in bloomy
- PWY-5005	with W = 437 -> more abundant in washed
- P164-PWY	with W = 429 -> more abundant in washed
- PWY-6107	with W = 424 -> more abundant in washed
- PWY-6942	with W = 421 -> more abundant in washed
- PWY-6629	with W = 419 -> more abundant in washed